# Experiments playground notebook
Use this notebook to run helper code and to test python cdk code excerpts

In [ ]:
%pip install boto3

## Change capacity mode for dynamo db
Use this to maange costs after the stack deployment. Provisioned capacity setting to support performance requirements for RTB is the major cost contributor in this stack.

In [ ]:
# use python sdk to connect to a profile session
import boto3
profile_name = 'rtb'
root_stack_name = 'rtbkit-bkr-dev'
session = boto3.session.Session(profile_name=profile_name)
# create a dynamodb client
dynamodb = session.client('dynamodb')
# list all tables
try:
    tables = dynamodb.list_tables()['TableNames']
    # if table name starts with root_stack_name, check the capacity mode, if the capacity mode is provisioned change it to on-demand
    for table in tables:
        if table.startswith(root_stack_name):
            response = dynamodb.describe_table(TableName=table)['Table']
            print(response)
            if 'BillingModeSummary' in response:
                billing_mode = response['BillingModeSummary']['BillingMode']
                if billing_mode == 'PROVISIONED':
                    print("Table", table, "is using provisioned capacity")
                    try:
                        uresponse = dynamodb.update_table(TableName=table,BillingMode='PAY_PER_REQUEST')
                        print(f"Table {table} updated successfully to on-demand mode.")
                    except Exception as e:
                        print(f"Error updating table: {str(e)}")
                elif billing_mode == 'PAY_PER_REQUEST':
                    print("Table", table, "is using on-demand capacity")
            else:
                # If BillingModeSummary is not present, the table is using provisioned capacity
                print("Table", table, "is using provisioned capacity")
                try:
                    uresponse = dynamodb.update_table(TableName=table,BillingMode='PAY_PER_REQUEST')
                    print(f"Table {table} updated successfully to on-demand mode.")
                except Exception as e:
                    print(f"Error updating table: {str(e)}")
except Exception as e:
    print(f"Error describing table: {str(e)}")

## Change kinesis shard count and capacity mode

In [ ]:
# list all kinesis streams whose name starts with a prefix
from time import sleep
kinesis_client = session.client('kinesis')
response = kinesis_client.list_streams()
# print(response)
stream_arns = [item['StreamARN'] for item in response['StreamSummaries'] if root_stack_name in item['StreamARN']]
print(stream_arns)
# check the capacity mode of each stream
for stream_arn in stream_arns:
    response = kinesis_client.describe_stream_summary(StreamARN=stream_arn)
    capacity_mode = response['StreamDescriptionSummary']['StreamModeDetails']['StreamMode']
    if capacity_mode == 'PROVISIONED':
        shards = response['StreamDescriptionSummary']['OpenShardCount']
        print(f"Stream: {stream_arn}, Capacity Mode: {capacity_mode}", f"Shards: {shards}")
        # first set shard to half of existing and then set shard count to 200
        if shards >=200:
            try:
                kinesis_client.update_shard_count(StreamARN=stream_arn, TargetShardCount=200, ScalingType='UNIFORM_SCALING')
            except kinesis_client.exceptions.InvalidArgumentException as e:
                if "UpdateShardCount cannot scale down below half your current open shard count" in str(e):
                    print(f"Error updating shard count for stream: {stream_arn}, error: {e}")
                    half_shards = int(shards / 2)
                    kinesis_client.update_shard_count(StreamARN=stream_arn, TargetShardCount=half_shards, ScalingType='UNIFORM_SCALING')
                    while True:
                        response = kinesis_client.describe_stream_summary(StreamARN=stream_arn)
                        shard_count = response['StreamDescriptionSummary']['OpenShardCount']
                        if shard_count == half_shards:
                            print(f"Updated shard count to {shard_count} for stream: {stream_arn}")
                            break
                        print(f"Waiting for shard count update for stream: {stream_arn}, current shard count: {shard_count}")
                        sleep(10)
            else:
                # keep checking the shard count until it's updated
                while True:
                    response = kinesis_client.describe_stream_summary(StreamARN=stream_arn)
                    shard_count = response['StreamDescriptionSummary']['OpenShardCount']
                    if shard_count == 200:
                        print(f"Updated shard count to {shard_count} for stream: {stream_arn}")
                        break
                    print(f"Waiting for shard count update for stream: {stream_arn}, current shard count: {shard_count}")
                    sleep(10)
            # let the previous shard count update complete and the stream in "ACTIVE" status
            sleep(10)
            # change scaling mode to on demand
            kinesis_client.update_stream_mode(
                StreamARN=kinesis_client.describe_stream(StreamARN=stream_arn)['StreamDescription']['StreamARN'],
                StreamModeDetails={'StreamMode': 'ON_DEMAND'}
            )
        else:
            kinesis_client.update_stream_mode(
                StreamARN=kinesis_client.describe_stream(StreamARN=stream_arn)['StreamDescription']['StreamARN'],
                StreamModeDetails={'StreamMode': 'ON_DEMAND'}
            )
            # let the previous shard count update complete and the stream in "ACTIVE" status
            sleep(10)
            capacity_mode = response['StreamDescriptionSummary']['StreamModeDetails']['StreamMode']
            if capacity_mode == 'ON_DEMAND':
                print(f"Stream: {stream_arn}, Updated Capacity Mode: {capacity_mode}")
    else:
        print(f"Stream: {stream_arn}, Capacity Mode: {capacity_mode}")

    

In [ ]:
""